In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import random
import pandas as pd
import shutil
from bayes_opt import BayesianOptimization


# ==== 基因演算法會用到的函式 ====
def fitFunc_2(x):                                                            # 適應度函數
    S = np.zeros((NUM_JOB, NUM_MACHINE))    # S[i][j] = Starting time of job i at machine j
    C = np.zeros((NUM_JOB, NUM_MACHINE))    # C[i][j] = Completion time of job i at machine j    
    B = np.zeros(NUM_MACHINE, dtype=int)    # B[j] = Available time of machine j  
    opJob = np.zeros(NUM_JOB, dtype=int)    # opJob[i] = current operation ID of job i
    
    for i in range(NUM_BIT):
        m = mOrder[x[i]][opJob[x[i]]]
        if opJob[x[i]] != 0:
            S[x[i]][m] = max([B[m], C[x[i]][mOrder[x[i]][opJob[x[i]]-1]]])
        else:
            S[x[i]][m] = B[m]        
        C[x[i]][m] = B[m] = S[x[i]][m] + pTime[x[i]][opJob[x[i]]]
        opJob[x[i]] += 1
    return S, C, B, -max(B)

def SCB_xlsx(Start, Comp):
    Total = []
    L = []
    for i in range(len(Start)):
        Q = []
        L.append("Job" + str(i+1))
        for j in range(len(Start[i])):
            Q.append(Start[i][j])
            Q.append(Comp[i][j])
        Total.append(Q)

    Q = []
    for j in range(len(Start[0])):
        Q.append("StartTime_M" + str(j+1))
        Q.append("CompletionTime_M" + str(j+1))

    df = pd.DataFrame(Total)
    df.index = L  # 设置行索引标签
    df.columns = Q  # 设置列索引标签
    with pd.ExcelWriter(new_folder_path +'best_solution.xlsx', mode='a') as writer:
        df.to_excel(writer, sheet_name='Time')
    
def gantt(s, t):
    fig, ax = plt.subplots(figsize=(10, 5)) #圖片大小
    ax.set_ylim(0, len(s)) #y大小
    
    s = np.transpose(s)  # 將 start 轉置
    t = np.transpose(t)  # 將 end 轉置
    
    for i in range(len(s)): #一條甘特圖
        #start_times = data[i][::2]
        #end_times = data[i][1::2]
        start_nums = s[i]
        end_nums = t[i]
        for j in range(len(start_nums)): #一段時間段
            start = start_nums[j]
            end = end_nums[j]
            ax.barh(i, end - start, left=start, height=0.9, align='center', alpha=0.8) #設定左右與長度
            # ax.annotate(str(start), xy=(start, i), xytext=(start, i-0.15), ha='center', va='top') #標注起始時間
            # ax.annotate(str(end), xy=(end, i), xytext=(end, i-0.15), ha='center', va='top') #標注結束時間

    ax.set_yticks(range(len(s))) #x軸範圍
    ax.set_yticklabels(['Machine{}'.format(i+1) for i in range(len(s))]) #job標注
    ax.set_xlabel('Time')
    plt.savefig(new_folder_path + "gantt.png")
    #plt.show()

def Sequence_xlsx(Start):
    TStart = Start.T
    Machine_order = []
    Seq = []
    Mac = []
    x = 0
    for i in range(len(TStart[0])):
        Seq.append("Seq" + str(i+1))
    for cards in TStart:
        x = x+1
        sorted_cards = [card[0] for card in sorted(enumerate(cards, 1), key=lambda x: x[1])]
        Machine_order.append(sorted_cards)
        Mac.append("Machine" + str(x))
    df = pd.DataFrame(Machine_order)
    df.index = Mac
    df.columns = Seq   
    with pd.ExcelWriter(new_folder_path +'best_solution.xlsx', mode='a') as writer:
        df.to_excel(writer, sheet_name='Sequence')

def transform_to_order(x):                                                 # 產生用在flexsim上的排程solution
    L = []
    for i in range(NUM_JOB):
        L.append([])
    for i in range(len(x)):
        L[x[i]].append(i+1)
    return L

def mkdir():                                                               # 創建新資料夾
    current_path = os.getcwd()              # 獲取當前.py文件的路徑
    new_folder_name = file_lname + "_solution"                                      # 新資料夾的名稱
    new_folder_path = os.path.join(current_path, new_folder_name)          # 新資料夾的完整路徑
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    return new_folder_path + '/'

def order_to_xlsx(x, y):                                                   # 用在flexsim上的排程輸出成excel檔
    data = transform_to_order(x)                                           # 假設你的二維列表是這樣：
    columns = ['Operation' + str(i + 1) for i in range(NUM_MACHINE)]                # 創建列名稱和行名稱
    index = ['Job' + str(i + 1) for i in range(NUM_JOB)]                        # 將二維列表轉換為 pandas DataFrame，並指定列名稱和行名稱
    df = pd.DataFrame(data, columns=columns, index=index)
    output_name = new_folder_path  + file_lname  + "_solution_" + "_result_" + str(-y) + ".xlsx"
    df1 = source(x)
    df2 = pd.DataFrame({'group_num': [NUM_CHROME], 'crossover_rate': [Pc], 'mutation_rate': [Pm], 'result': [-y]})
    with pd.ExcelWriter(output_name) as writer:
        df.to_excel(writer, sheet_name='Rank')                                         # 寫入 Excel 文件
        df1.to_excel(writer, sheet_name='Source')   
        df2.to_excel(writer, sheet_name='Parameter')   

def source(x):                                                             # 用在flexsim上的source輸出成excel檔
    job_set = set()
    Arrival_time = []
    ItemName = []
    ItemType = []
    Quantity = []
    row_names = []   # 創建一個空列表來存放行名稱
    id = 0
    for i in range(len(x)):
        if x[i] not in job_set:
            job_set.add(x[i])
            Arrival_time.append(0)
            ItemName.append('Product_' + str(x[i] + 1))
            ItemType.append(x[i] + 1)
            Quantity.append(1)
            id = id + 1
            row_names.append('Arrival' + str(id))   # 添加行名稱到列表中
            
    data = {
        'ArrivalTime': Arrival_time, 
        'ItemName': ItemName, 
        'ItemType': ItemType, 
        'Quantity': Quantity
    }

    df = pd.DataFrame(data, index=row_names)   # 將行名稱傳遞給 `pd.DataFrame`
    return df

def route_xlsx():                                                          # 用在flexsim上的source輸出成excel檔
    data = []
    L = []
    b = []
    for j in range(len(pTime[0]) + 1):
        b.append("Operation" + str(j+1))
    for i in range(len(pTime)):
        order = str(i+1)
        Route = "Route_" + order
        SetupTime = "SetupTime_" + order
        PTime = "pTime_" + order
        L.append(Route)
        L.append(SetupTime)
        L.append(PTime)
        data.append([])
        data.append([])
        data.append([])
    for i in range(len(pTime)):
        for j in range(len(pTime[i])):
            data[3*i].append('Buffer_' + str(mOrder[i][j]+1))
            data[3*i + 1].append(0)
            data[3*i + 2].append(pTime[i][j])
    for i in range(len(pTime)):
        data[3*i].append('Sink')
        data[3*i + 1].append(0)
        data[3*i + 2].append(0)
    df = pd.DataFrame(data, index = L, columns = b).T
    output_name = new_folder_path + file_lname + "_route.xlsx"
    df.to_excel(output_name)
  
def initPop():                                                             # 初始化群體
    p = []
    for i in range(NUM_CHROME) :        
        a = []
        for j in range(NUM_JOB):
            for k in range(NUM_MACHINE):
                a.append(j)
        np.random.shuffle(a)
        p.append(a)
    return p

def fitFunc(x):                                                            # 適應度函數
    S = np.zeros((NUM_JOB, NUM_MACHINE))    # S[i][j] = Starting time of job i at machine j
    C = np.zeros((NUM_JOB, NUM_MACHINE))    # C[i][j] = Completion time of job i at machine j    
    B = np.zeros(NUM_MACHINE, dtype=int)    # B[j] = Available time of machine j  
    opJob = np.zeros(NUM_JOB, dtype=int)    # opJob[i] = current operation ID of job i
    
    for i in range(NUM_BIT):
        m = mOrder[x[i]][opJob[x[i]]]
        if opJob[x[i]] != 0:
            S[x[i]][m] = max([B[m], C[x[i]][mOrder[x[i]][opJob[x[i]]-1]]])
        else:
            S[x[i]][m] = B[m]        
        C[x[i]][m] = B[m] = S[x[i]][m] + pTime[x[i]][opJob[x[i]]]
        opJob[x[i]] += 1
    return -max(B)           # 因為是最小化問題

def evaluatePop(p):                                                        # 評估群體之適應度
    return [fitFunc(p[i]) for i in range(len(p))]

def selection(p, p_fit):                                                   # 用二元競爭式選擇法來挑父母
    a = []
    for i in range(NUM_PARENT):
        [j, k] = np.random.choice(NUM_CHROME, 2, replace=False)  # 任選兩個index
        if p_fit[j] > p_fit[k] :                      # 擇優
            a.append(p[j].copy())
        else:
            a.append(p[k].copy())
    return a

def crossover_one_point(p):                                                # 用單點交配來繁衍子代 (new)
    a = []

    for i in range(NUM_CROSSOVER) :
        c = np.random.randint(1, NUM_BIT)      		  # 隨機找出單點(不包含0)
        [j, k] = np.random.choice(NUM_PARENT, 2, replace=False)  # 任選兩個index

        child1, child2 = p[j].copy(), p[k].copy()
        remain1, remain2 = list(p[j].copy()), list(p[k].copy())     # 存還沒被用掉的城市

        for m in range(NUM_BIT):
            if m < c :
                remain2.remove(child1[m])   # 砍掉 remain2 中的值是 child1[m]
                remain1.remove(child2[m])   # 砍掉 remain1 中的值是 child2[m]

        t = 0
        for m in range(NUM_BIT):
            if m >= c :
                child1[m] = remain2[t]
                child2[m] = remain1[t]
                t += 1

        a.append(child1)
        a.append(child2)
    return a

def crossover_uniform(p):                                                  # 用均勻交配來繁衍子代 (new)
    a = []
    for i in range(NUM_CROSSOVER) :
        mask = np.random.randint(2, size=NUM_BIT)
        [j, k] = np.random.choice(NUM_PARENT, 2, replace=False)  # 任選兩個index
        child1, child2 = p[j].copy(), p[k].copy()
        remain1, remain2 = list(p[j].copy()), list(p[k].copy())     # 存還沒被用掉的城市
        for m in range(NUM_BIT):
            if mask[m] == 1 :
                remain2.remove(child1[m])   # 砍掉 remain2 中的值是 child1[m]
                remain1.remove(child2[m])   # 砍掉 remain1 中的值是 child2[m]
        t = 0
        for m in range(NUM_BIT):
            if mask[m] == 0 :
                child1[m] = remain2[t]
                child2[m] = remain1[t]
                t += 1
        a.append(child1)
        a.append(child2)
    return a

def mutation(p):                                                        # 突變
    for _ in range(NUM_MUTATION) :
        if NUM_CROSSOVER_2 == 0: 
            break
        row = np.random.randint(NUM_CROSSOVER_2)    # 任選一個染色體
        [j, k] = np.random.choice(NUM_BIT, 2, replace=False)  # 任選兩個基因
        p[row][j], p[row][k] = p[row][k], p[row][j]       # 此染色體的兩基因互換

def sortChrome(a, a_fit):	                                               # a的根據a_fit由大排到小
    a_index = range(len(a))                         # 產生 0, 1, 2, ..., |a|-1 的 list    
    a_fit, a_index = zip(*sorted(zip(a_fit,a_index), reverse=True)) # a_index 根據 a_fit 的大小由大到小連動的排序
    return [a[i] for i in a_index], a_fit           # 根據 a_index 的次序來回傳 a，並把對應的 fit 回傳

def replace(p, p_fit, a, a_fit):                                           # 適者生存
    b = np.concatenate((p,a), axis=0)               # 把本代 p 和子代 a 合併成 b
    b_fit = p_fit + a_fit                           # 把上述兩代的 fitness 合併成 b_fit
    b, b_fit = sortChrome(b, b_fit)                 # b 和 b_fit 連動的排序  
    return b[:NUM_CHROME], list(b_fit[:NUM_CHROME]) # 回傳 NUM_CHROME 個為新的一個世代

def readfile():                                                            #讀取檔案用在GA_solver
    with open(file_name, 'r') as f:
        file_list = [list(map(lambda x: int(x) ,line.split())) for line in f]
    return file_list

def iter_files(directory):                                                 #找到資料夾路徑
    for filename in os.listdir(directory):
        yield os.path.join(directory, filename)

def GA_solver(group_num, crossover_rate, mutation_rate):                   #GA演算法
    global NUM_CHROME, Pc, Pm, NUM_PARENT, NUM_CROSSOVER, NUM_CROSSOVER_2, NUM_MUTATION, NUM_ITERATION, best_x, best_y
    NUM_CHROME = int(group_num)
    Pc = crossover_rate
    Pm = mutation_rate
    #if NUM_ITERATION >= 2000:
    #   Pm = 2 * Pm
    NUM_PARENT = NUM_CHROME
    NUM_CROSSOVER = int(Pc * NUM_CHROME / 2)          # 交配的次數
    NUM_CROSSOVER_2 = NUM_CROSSOVER * 2               # 上數的兩倍
    NUM_MUTATION = int(Pm * NUM_CHROME * NUM_BIT)     # 突變的次數# === Step 3-2. NUM_BIT 要修改成 3 x 3 ===
    #print( Pc, Pm, GA_ITERATION, NUM_PARENT, NUM_CROSSOVER, NUM_CROSSOVER_2, NUM_MUTATION, NUM_ITERATION)
    # ==== 主程式 ==== 
    pop = initPop()                                 # 初始化 pop
    pop_fit = evaluatePop(pop)                      # 算 pop 的 fit
    best_outputs = []                               # 用此變數來紀錄每一個迴圈的最佳解 (new)
    best_outputs.append(np.max(pop_fit))            # 存下初始群體的最佳解
    mean_outputs = []                               # 用此變數來紀錄每一個迴圈的平均解 (new)
    mean_outputs.append(np.average(pop_fit))        # 存下初始群體的最佳解

    for i in range(NUM_ITERATION) :
        parent = selection(pop, pop_fit)            # 挑父母
        offspring = crossover_one_point(parent)     # 單點交配
        mutation(offspring)                         # 突變
        offspring_fit = evaluatePop(offspring)      # 算子代的 fit
        if NUM_CROSSOVER_2 != 0:
            pop, pop_fit = replace(pop, pop_fit, offspring, offspring_fit)    # 取代
        best_outputs.append(np.max(pop_fit))        # 存下這次的最佳解
        mean_outputs.append(np.average(pop_fit))    # 存下這次的平均解
        print('iteration %d: y = %d'	%(i, -pop_fit[0]))     # fit 改負的
       #print('iteration %d: x = %s, y = %d'	%(i, pop[0], -pop_fit[0]))     # fit 改負的
    #return best_outputs, mean_outputs, pop[0], -pop_fit[0]
    order_to_xlsx(pop[0], pop_fit[0])
    if abs(pop_fit[0]) < abs(best_y[0]):
        best_y = pop_fit
        best_x = pop 
    plt.plot(best_outputs)
    plt.plot(mean_outputs)
    plt.xlabel("Iteration")
    plt.ylabel("Fitness")
    plt.show()
    return pop_fit[0]


file_name = input("請輸入問題的資料夾路徑：")

# ==== 參數設定(與問題相關) ====
best_y = [100000, 1000000]
best_x = []
file_list = readfile() 
last_part = file_name.rsplit('/', 1)[-1]           # 這裡先取得 '/' 之後的所有字符
file_lname = last_part.split('.')[0]               # 然後取得 '.' 之前的所有字符 
new_folder_path = mkdir()              
pTime = [[file_list[i][j] for j in range(len(file_list[i])) if j % 2 == 1] for i in range(1, len(file_list))]
mOrder = [[file_list[i][j] for j in range(len(file_list[i])) if j % 2 == 0] for i in range(1, len(file_list))]
route_xlsx()


NUM_JOB = file_list[0][0]           
NUM_MACHINE = file_list[0][1] 
NUM_BIT = NUM_JOB * NUM_MACHINE                   # 染色體長度 # === Step 3-1. 編碼是 000111222 的排列 ===

# ==== 參數設定(與演算法相關) ====
NUM_ITERATION = 30000                               # 世代數(迴圈數)
NUM_CHROME = 63.584044712736016                                    # 染色體個數
Pc = 0.8471666005438594                                          # 交配率 (代表共執行Pc*NUM_CHROME/2次交配)
Pm = 0.008647728413878389                                          # 突變率 (代表共要執行Pm*NUM_CHROME*NUM_BIT次突變)
NUM_PARENT = 0                                    # 父母的個數
NUM_CROSSOVER = 0                                 # 交配的次數
NUM_CROSSOVER_2 = 0                               # 上數的兩倍
NUM_MUTATION = 0                                  # 突變的次數 
np.random.seed(0)                                 # 若要每次跑得都不一樣的結果，就把這行註解掉    

GA_solver(242.1,0.7186,0.01422)
print(file_name)
print(best_x, best_y)

Start, Completion, Available, sol = fitFunc_2(best_x[0])
SCB_xlsx(Start, Completion)
Sequence_xlsx(Start)
gantt(Start, Completion)


#best_outputs, mean_outputs, sol_x, sol_y = GA_solver(0.4, 0.7);
#print(sol_x, sol_y, Pc, Pm)        
# 畫圖


請輸入問題的資料夾路徑：/home/yarikama/Desktop/Simulation/r20_15/rcmax_20_15_8.txt
iteration 0: y = 4699
iteration 1: y = 4651
iteration 2: y = 4626
iteration 3: y = 4558
iteration 4: y = 4403
iteration 5: y = 4403
iteration 6: y = 4403
iteration 7: y = 4289
iteration 8: y = 4289
iteration 9: y = 4289
iteration 10: y = 4289
iteration 11: y = 4289
iteration 12: y = 4289
iteration 13: y = 4289
iteration 14: y = 4289
iteration 15: y = 4245
iteration 16: y = 4245
iteration 17: y = 4245
iteration 18: y = 4245
iteration 19: y = 4172
iteration 20: y = 4172
iteration 21: y = 4151
iteration 22: y = 4093
iteration 23: y = 4093
iteration 24: y = 4093
iteration 25: y = 4074
iteration 26: y = 4074
iteration 27: y = 4074
iteration 28: y = 4074
iteration 29: y = 3978
iteration 30: y = 3978
iteration 31: y = 3978
iteration 32: y = 3978
iteration 33: y = 3978
iteration 34: y = 3978
iteration 35: y = 3978
iteration 36: y = 3937
iteration 37: y = 3937
iteration 38: y = 3937
iteration 39: y = 3937
iteration 40: y = 3

iteration 344: y = 3402
iteration 345: y = 3402
iteration 346: y = 3402
iteration 347: y = 3402
iteration 348: y = 3402
iteration 349: y = 3402
iteration 350: y = 3372
iteration 351: y = 3372
iteration 352: y = 3372
iteration 353: y = 3372
iteration 354: y = 3372
iteration 355: y = 3372
iteration 356: y = 3372
iteration 357: y = 3372
iteration 358: y = 3372
iteration 359: y = 3372
iteration 360: y = 3372
iteration 361: y = 3372
iteration 362: y = 3372
iteration 363: y = 3372
iteration 364: y = 3372
iteration 365: y = 3372
iteration 366: y = 3372
iteration 367: y = 3372
iteration 368: y = 3372
iteration 369: y = 3372
iteration 370: y = 3372
iteration 371: y = 3372
iteration 372: y = 3372
iteration 373: y = 3372
iteration 374: y = 3372
iteration 375: y = 3372
iteration 376: y = 3372
iteration 377: y = 3372
iteration 378: y = 3372
iteration 379: y = 3372
iteration 380: y = 3372
iteration 381: y = 3372
iteration 382: y = 3372
iteration 383: y = 3372
iteration 384: y = 3372
iteration 385: y

iteration 686: y = 3276
iteration 687: y = 3276
iteration 688: y = 3276
iteration 689: y = 3276
iteration 690: y = 3276
iteration 691: y = 3276
iteration 692: y = 3276
iteration 693: y = 3276
iteration 694: y = 3276
iteration 695: y = 3276
iteration 696: y = 3276
iteration 697: y = 3276
iteration 698: y = 3276
iteration 699: y = 3276
iteration 700: y = 3276
iteration 701: y = 3276
iteration 702: y = 3276
iteration 703: y = 3276
iteration 704: y = 3276
iteration 705: y = 3276
iteration 706: y = 3276
iteration 707: y = 3276
iteration 708: y = 3276
iteration 709: y = 3276
iteration 710: y = 3276
iteration 711: y = 3276
iteration 712: y = 3276
iteration 713: y = 3276
iteration 714: y = 3276
iteration 715: y = 3276
iteration 716: y = 3276
iteration 717: y = 3276
iteration 718: y = 3276
iteration 719: y = 3276
iteration 720: y = 3276
iteration 721: y = 3276
iteration 722: y = 3276
iteration 723: y = 3276
iteration 724: y = 3276
iteration 725: y = 3276
iteration 726: y = 3274
iteration 727: y

iteration 1028: y = 3228
iteration 1029: y = 3228
iteration 1030: y = 3228
iteration 1031: y = 3228
iteration 1032: y = 3228
iteration 1033: y = 3228
iteration 1034: y = 3228
iteration 1035: y = 3228
iteration 1036: y = 3228
iteration 1037: y = 3228
iteration 1038: y = 3228
iteration 1039: y = 3228
iteration 1040: y = 3228
iteration 1041: y = 3228
iteration 1042: y = 3228
iteration 1043: y = 3228
iteration 1044: y = 3228
iteration 1045: y = 3228
iteration 1046: y = 3228
iteration 1047: y = 3228
iteration 1048: y = 3228
iteration 1049: y = 3228
iteration 1050: y = 3228
iteration 1051: y = 3228
iteration 1052: y = 3228
iteration 1053: y = 3228
iteration 1054: y = 3228
iteration 1055: y = 3228
iteration 1056: y = 3228
iteration 1057: y = 3228
iteration 1058: y = 3228
iteration 1059: y = 3228
iteration 1060: y = 3228
iteration 1061: y = 3228
iteration 1062: y = 3228
iteration 1063: y = 3228
iteration 1064: y = 3228
iteration 1065: y = 3228
iteration 1066: y = 3228
iteration 1067: y = 3228


iteration 1356: y = 3173
iteration 1357: y = 3173
iteration 1358: y = 3173
iteration 1359: y = 3173
iteration 1360: y = 3173
iteration 1361: y = 3173
iteration 1362: y = 3173
iteration 1363: y = 3173
iteration 1364: y = 3173
iteration 1365: y = 3173
iteration 1366: y = 3173
iteration 1367: y = 3173
iteration 1368: y = 3173
iteration 1369: y = 3173
iteration 1370: y = 3173
iteration 1371: y = 3173
iteration 1372: y = 3173
iteration 1373: y = 3173
iteration 1374: y = 3173
iteration 1375: y = 3173
iteration 1376: y = 3173
iteration 1377: y = 3173
iteration 1378: y = 3173
iteration 1379: y = 3173
iteration 1380: y = 3173
iteration 1381: y = 3173
iteration 1382: y = 3173
iteration 1383: y = 3173
iteration 1384: y = 3173
iteration 1385: y = 3173
iteration 1386: y = 3173
iteration 1387: y = 3173
iteration 1388: y = 3173
iteration 1389: y = 3173
iteration 1390: y = 3173
iteration 1391: y = 3173
iteration 1392: y = 3173
iteration 1393: y = 3173
iteration 1394: y = 3173
iteration 1395: y = 3173


...

iteration 19072: y = 3036
iteration 19073: y = 3036
iteration 19074: y = 3036
iteration 19075: y = 3036
iteration 19076: y = 3036
iteration 19077: y = 3036
iteration 19078: y = 3036
iteration 19079: y = 3036
iteration 19080: y = 3036
iteration 19081: y = 3036
iteration 19082: y = 3036
iteration 19083: y = 3036
iteration 19084: y = 3036
iteration 19085: y = 3036
iteration 19086: y = 3036
iteration 19087: y = 3036
iteration 19088: y = 3036
iteration 19089: y = 3036
iteration 19090: y = 3036
iteration 19091: y = 3036
iteration 19092: y = 3036
iteration 19093: y = 3036
iteration 19094: y = 3036
iteration 19095: y = 3036
iteration 19096: y = 3036
iteration 19097: y = 3036
iteration 19098: y = 3036
iteration 19099: y = 3036
iteration 19100: y = 3036
iteration 19101: y = 3036
iteration 19102: y = 3036
iteration 19103: y = 3036
iteration 19104: y = 3036
iteration 19105: y = 3036
iteration 19106: y = 3036
iteration 19107: y = 3036
iteration 19108: y = 3036
iteration 19109: y = 3036
iteration 19

iteration 19388: y = 3036
iteration 19389: y = 3036
iteration 19390: y = 3036
iteration 19391: y = 3036
iteration 19392: y = 3036
iteration 19393: y = 3036
iteration 19394: y = 3036
iteration 19395: y = 3036
iteration 19396: y = 3036
iteration 19397: y = 3036
iteration 19398: y = 3036
iteration 19399: y = 3036
iteration 19400: y = 3036
iteration 19401: y = 3036
iteration 19402: y = 3036
iteration 19403: y = 3036
iteration 19404: y = 3036
iteration 19405: y = 3036
iteration 19406: y = 3036
iteration 19407: y = 3036
iteration 19408: y = 3036
iteration 19409: y = 3036
iteration 19410: y = 3036
iteration 19411: y = 3036
iteration 19412: y = 3036
iteration 19413: y = 3036
iteration 19414: y = 3036
iteration 19415: y = 3036
iteration 19416: y = 3036
iteration 19417: y = 3036
iteration 19418: y = 3036
iteration 19419: y = 3036
iteration 19420: y = 3036
iteration 19421: y = 3036
iteration 19422: y = 3036
iteration 19423: y = 3036
iteration 19424: y = 3036
iteration 19425: y = 3036
iteration 19

iteration 19704: y = 3036
iteration 19705: y = 3036
iteration 19706: y = 3036
iteration 19707: y = 3036
iteration 19708: y = 3036
iteration 19709: y = 3036
iteration 19710: y = 3036
iteration 19711: y = 3036
iteration 19712: y = 3036
iteration 19713: y = 3036
iteration 19714: y = 3036
iteration 19715: y = 3036
iteration 19716: y = 3036
iteration 19717: y = 3036
iteration 19718: y = 3036
iteration 19719: y = 3036
iteration 19720: y = 3036
iteration 19721: y = 3036
iteration 19722: y = 3036
iteration 19723: y = 3036
iteration 19724: y = 3036
iteration 19725: y = 3036
iteration 19726: y = 3036
iteration 19727: y = 3036
iteration 19728: y = 3036
iteration 19729: y = 3036
iteration 19730: y = 3036
iteration 19731: y = 3036
iteration 19732: y = 3036
iteration 19733: y = 3036
iteration 19734: y = 3036
iteration 19735: y = 3036
iteration 19736: y = 3036
iteration 19737: y = 3036
iteration 19738: y = 3036
iteration 19739: y = 3036
iteration 19740: y = 3036
iteration 19741: y = 3036
iteration 19

iteration 20020: y = 3036
iteration 20021: y = 3036
iteration 20022: y = 3036
iteration 20023: y = 3036
iteration 20024: y = 3036
iteration 20025: y = 3036
iteration 20026: y = 3036
iteration 20027: y = 3036
iteration 20028: y = 3036
iteration 20029: y = 3036
iteration 20030: y = 3036
iteration 20031: y = 3036
iteration 20032: y = 3036
iteration 20033: y = 3036
iteration 20034: y = 3036
iteration 20035: y = 3036
iteration 20036: y = 3036
iteration 20037: y = 3036
iteration 20038: y = 3036
iteration 20039: y = 3036
iteration 20040: y = 3036
iteration 20041: y = 3036
iteration 20042: y = 3036
iteration 20043: y = 3036
iteration 20044: y = 3036
iteration 20045: y = 3036
iteration 20046: y = 3036
iteration 20047: y = 3036
iteration 20048: y = 3036
iteration 20049: y = 3036
iteration 20050: y = 3036
iteration 20051: y = 3036
iteration 20052: y = 3036
iteration 20053: y = 3036
iteration 20054: y = 3036
iteration 20055: y = 3036
iteration 20056: y = 3036
iteration 20057: y = 3036
iteration 20

iteration 20336: y = 3036
iteration 20337: y = 3036
iteration 20338: y = 3036
iteration 20339: y = 3036
iteration 20340: y = 3036
iteration 20341: y = 3036
iteration 20342: y = 3036
iteration 20343: y = 3036
iteration 20344: y = 3036
iteration 20345: y = 3036
iteration 20346: y = 3036
iteration 20347: y = 3036
iteration 20348: y = 3036
iteration 20349: y = 3036
iteration 20350: y = 3036
iteration 20351: y = 3036
iteration 20352: y = 3036
iteration 20353: y = 3036
iteration 20354: y = 3036
iteration 20355: y = 3036
iteration 20356: y = 3036
iteration 20357: y = 3036
iteration 20358: y = 3036
iteration 20359: y = 3036
iteration 20360: y = 3036
iteration 20361: y = 3036
iteration 20362: y = 3036
iteration 20363: y = 3036
iteration 20364: y = 3036
iteration 20365: y = 3036
iteration 20366: y = 3036
iteration 20367: y = 3036
iteration 20368: y = 3036
iteration 20369: y = 3036
iteration 20370: y = 3036
iteration 20371: y = 3036
iteration 20372: y = 3036
iteration 20373: y = 3036
iteration 20